In [1]:
import torch
import torch.utils.data
from TDNN_gpu import TDNN
import torch.optim as optim
import numpy as np
import torch.nn as nn
from TDNN_gpu import StatsPooling
from TDNN_gpu import FullyConnected
import torch.nn.functional as F
import os
from scipy.io import wavfile
from python_speech_features import mfcc, logfbank
from tqdm import tqdm 
import pickle
import time

 ## Data preperation

In [2]:
def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]

In [3]:
def to_categorical(y, num_classes):
    """ 1-hot encodes a tensor """
    return np.eye(num_classes, dtype='uint8')[y]

In [4]:
clases = ['Trump','Macron','Gaga','Clinton']

In [5]:
# X = []
# y = []
# max_total_context = 400
# _min,_max = float('inf'),-float('inf')

# for wav_name in os.listdir('train_cleaned'):
#     if wav_name.endswith('wav'):
#         print (wav_name)
#         rate, wav = wavfile.read(r'train_cleaned/'+wav_name)
#         for chunked_wav in tqdm(chunks(wav,int(len(wav)/4))):
#             X_sample = mfcc(chunked_wav,samplerate= rate,numcep=24
#                    ,nfilt=26,nfft=1024)
#             _min = min(np.amin(X_sample),_min)
#             _max = max(np.amax(X_sample),_max)
            
#             for chunked_X_sample in list(chunks(X_sample,max_total_context)):
#                 if len(chunked_X_sample) == max_total_context:
#                     X.append(chunked_X_sample)
#                     y.append(to_categorical(clases.index(wav_name.split('.wav')[0]),len(clases)))
#                 else:
#                     print ('discarded box')
# X = (X - _min) / (_max-_min)
# with open('X_data.pkl','wb') as f:
#     pickle.dump((X,y),f)

In [6]:
with open('X_data.pkl','rb') as f:
    X,y =  pickle.load(f)

In [7]:
# gg = [0,0,0]*8
#in the input LINES we keep the MFCC's values
#in the input ROWS we keep the diffrent frames
# zz = [1.0,2.0,3.0]*8
# a= [gg,gg,zz,zz,zz,zz,zz,zz,zz]*30
# train_data = []
# x_data = np.array([a]).astype(np.double)
# labels = ['a','b','c','d','e']
# for i in range(len(x_data)):
#     train_data.append([x_data[i], labels[i]])

trainloader = torch.utils.data.DataLoader(list(zip(X,y)), shuffle=True, batch_size=5)

In [8]:
# device = torch.device('cuda:2' if torch.cuda.is_available() else "cpu")

In [9]:
# device = "cpu"

In [10]:
context = [-2,2]
input_dim = 24
output_dim = 512
net1 = TDNN(context, input_dim, output_dim, full_context=True)

context = [-2,1,2]
input_dim = 512
output_dim = 512
net2 = TDNN(context, input_dim, output_dim, full_context=False)

context = [-3,1,3]
input_dim = 512
output_dim = 512
net3 = TDNN(context, input_dim, output_dim, full_context=False)

context = [1]
input_dim = 512
output_dim = 512
net4 = TDNN(context, input_dim, output_dim, full_context=False)

context = [1]
input_dim = 512
output_dim = 1500
net5 = TDNN(context, input_dim, output_dim, full_context=False)

SP = StatsPooling()
FC = FullyConnected()
Final = nn.Linear(512,4).double()
net = nn.Sequential(net1,net2,net3,net4,net5,SP,FC,Final)

In [11]:
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
criterion = nn.CrossEntropyLoss()

In [12]:
# if device:
#     net= net.to(device)
#     print("GPU")
# else:
#     print("NO GPU")

In [13]:
net.cuda()

Sequential(
  (0): TDNN()
  (1): TDNN()
  (2): TDNN()
  (3): TDNN()
  (4): TDNN()
  (5): StatsPooling()
  (6): FullyConnected(
    (hidden1): Linear(in_features=3000, out_features=512, bias=True)
    (hidden2): Linear(in_features=512, out_features=512, bias=True)
  )
  (7): Linear(in_features=512, out_features=4, bias=True)
)

In [33]:
epocs = 50
for j in range (epocs):
    for i, data in enumerate(trainloader, 0):
        a = time.time()
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs, labels = inputs.cuda(), labels.cuda()
        output= net(inputs).cuda()
        optimizer.zero_grad()
        loss = criterion(output, torch.max(labels, 1)[1])
        loss.backward()
        optimizer.step()
        b = time.time()
        print ("loss: {} step took {}".format(loss,b-a))
        print ("output:\n{} \n labels:\n{}".format(output,labels))
        torch.save(net, 'total_model')


mel size: 24
sequence length: 400
output shape: torch.Size([5, 396, 512])
mel size: 512
sequence length: 396
output shape: torch.Size([5, 392, 512])
mel size: 512
sequence length: 392
output shape: torch.Size([5, 386, 512])
mel size: 512
sequence length: 386
output shape: torch.Size([5, 385, 512])
mel size: 512
sequence length: 385
output shape: torch.Size([5, 384, 1500])
loss: 0.2348794427783556 step took 25.042848825454712
output:
tensor([[ 0.8693,  5.3925, -9.9476,  3.7670],
        [ 3.3334, -9.1446,  4.5785,  1.5976],
        [ 1.7789, -3.2525, -1.6036,  3.4541],
        [ 0.9113,  4.9453, -9.5045,  3.7482],
        [ 1.9215, -3.9279, -0.9050,  3.2931]], device='cuda:0',
       dtype=torch.float64, grad_fn=<AddmmBackward>) 
 labels:
tensor([[0, 1, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1],
        [0, 1, 0, 0],
        [0, 0, 0, 1]], device='cuda:0', dtype=torch.uint8)
mel size: 24
sequence length: 400
output shape: torch.Size([5, 396, 512])
mel size: 512
sequence length: 

        [1, 0, 0, 0]], device='cuda:0', dtype=torch.uint8)
mel size: 24
sequence length: 400
output shape: torch.Size([5, 396, 512])
mel size: 512
sequence length: 396
output shape: torch.Size([5, 392, 512])
mel size: 512
sequence length: 392
output shape: torch.Size([5, 386, 512])
mel size: 512
sequence length: 386
output shape: torch.Size([5, 385, 512])
mel size: 512
sequence length: 385
output shape: torch.Size([5, 384, 1500])
loss: 0.3427280095937383 step took 23.21104598045349
output:
tensor([[  3.1000, -10.9827,   7.3058,   0.8944],
        [  3.2889,  -7.8565,   3.9511,   0.8704],
        [  0.6841,   5.4732,  -9.1478,   3.0220],
        [  1.7026,  -3.5696,  -0.3649,   2.5215],
        [  0.5341,   5.5959,  -9.3034,   3.2146]], device='cuda:0',
       dtype=torch.float64, grad_fn=<AddmmBackward>) 
 labels:
tensor([[0, 0, 1, 0],
        [1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 0, 1],
        [0, 1, 0, 0]], device='cuda:0', dtype=torch.uint8)
mel size: 24
sequence lengt

        [0, 1, 0, 0]], device='cuda:0', dtype=torch.uint8)
mel size: 24
sequence length: 400
output shape: torch.Size([5, 396, 512])
mel size: 512
sequence length: 396
output shape: torch.Size([5, 392, 512])
mel size: 512
sequence length: 392
output shape: torch.Size([5, 386, 512])
mel size: 512
sequence length: 386
output shape: torch.Size([5, 385, 512])
mel size: 512
sequence length: 385
output shape: torch.Size([5, 384, 1500])
loss: 0.3375522130886253 step took 23.16482424736023
output:
tensor([[  0.4052,   7.2933, -11.0001,   3.2807],
        [  2.1499,  -5.3833,   1.3567,   2.1842],
        [  1.0428,   4.9820,  -8.7883,   2.7960],
        [  0.7819,   5.6622,  -9.5269,   3.1154],
        [  1.9120,  -4.5619,   0.5769,   2.3753]], device='cuda:0',
       dtype=torch.float64, grad_fn=<AddmmBackward>) 
 labels:
tensor([[0, 1, 0, 0],
        [0, 0, 0, 1],
        [0, 1, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 0, 1]], device='cuda:0', dtype=torch.uint8)
mel size: 24
sequence lengt

        [0, 0, 1, 0]], device='cuda:0', dtype=torch.uint8)
mel size: 24
sequence length: 400
output shape: torch.Size([5, 396, 512])
mel size: 512
sequence length: 396
output shape: torch.Size([5, 392, 512])
mel size: 512
sequence length: 392
output shape: torch.Size([5, 386, 512])
mel size: 512
sequence length: 386
output shape: torch.Size([5, 385, 512])
mel size: 512
sequence length: 385
output shape: torch.Size([5, 384, 1500])
loss: 0.11079418927128577 step took 23.209880590438843
output:
tensor([[  3.3965,  -4.7910,   0.8872,   0.6532],
        [  1.1676,   4.4756,  -8.2070,   2.5996],
        [  3.3626,  -5.2243,   1.2939,   0.7324],
        [  0.6597,   7.6716, -11.2901,   2.8966],
        [  1.1923,   5.6209,  -9.3043,   2.4751]], device='cuda:0',
       dtype=torch.float64, grad_fn=<AddmmBackward>) 
 labels:
tensor([[1, 0, 0, 0],
        [0, 1, 0, 0],
        [1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 1, 0, 0]], device='cuda:0', dtype=torch.uint8)
mel size: 24
sequence len

        [1, 0, 0, 0]], device='cuda:0', dtype=torch.uint8)
mel size: 24
sequence length: 400
output shape: torch.Size([5, 396, 512])
mel size: 512
sequence length: 396
output shape: torch.Size([5, 392, 512])
mel size: 512
sequence length: 392
output shape: torch.Size([5, 386, 512])
mel size: 512
sequence length: 386
output shape: torch.Size([5, 385, 512])
mel size: 512
sequence length: 385
output shape: torch.Size([5, 384, 1500])
loss: 0.08578391511188181 step took 23.051780462265015
output:
tensor([[  3.0293, -11.0060,   7.5520,   0.7222],
        [  2.9293, -11.2696,   7.7460,   0.9047],
        [ -0.2602,   7.7213, -11.1644,   3.6988],
        [  1.3049,  -4.9711,   1.2593,   2.7478],
        [  2.8588, -11.1152,   7.6212,   0.9452]], device='cuda:0',
       dtype=torch.float64, grad_fn=<AddmmBackward>) 
 labels:
tensor([[0, 0, 1, 0],
        [0, 0, 1, 0],
        [0, 1, 0, 0],
        [0, 0, 0, 1],
        [0, 0, 1, 0]], device='cuda:0', dtype=torch.uint8)
mel size: 24
sequence len

...

        [0, 0, 0, 1]], device='cuda:0', dtype=torch.uint8)
mel size: 24
sequence length: 400
output shape: torch.Size([5, 396, 512])
mel size: 512
sequence length: 396
output shape: torch.Size([5, 392, 512])
mel size: 512
sequence length: 392
output shape: torch.Size([5, 386, 512])
mel size: 512
sequence length: 386
output shape: torch.Size([5, 385, 512])
mel size: 512
sequence length: 385
output shape: torch.Size([5, 384, 1500])
loss: 0.1211241346154309 step took 22.609508752822876
output:
tensor([[  2.5605,  -3.2799,  -2.2694,   3.2271],
        [  2.3904,  10.5610, -16.2124,   3.0850],
        [  2.0651,  -1.8746,  -3.6724,   3.7217],
        [  2.6976,  10.3722, -16.0502,   2.7964],
        [  3.0270,   8.4418, -14.2247,   2.6463]], device='cuda:0',
       dtype=torch.float64, grad_fn=<AddmmBackward>) 
 labels:
tensor([[0, 0, 0, 1],
        [0, 1, 0, 0],
        [0, 0, 0, 1],
        [0, 1, 0, 0],
        [0, 1, 0, 0]], device='cuda:0', dtype=torch.uint8)
mel size: 24
sequence leng

        [0, 0, 0, 1]], device='cuda:0', dtype=torch.uint8)
mel size: 24
sequence length: 400
output shape: torch.Size([5, 396, 512])
mel size: 512
sequence length: 396
output shape: torch.Size([5, 392, 512])
mel size: 512
sequence length: 392
output shape: torch.Size([5, 386, 512])
mel size: 512
sequence length: 386
output shape: torch.Size([5, 385, 512])
mel size: 512
sequence length: 385
output shape: torch.Size([5, 384, 1500])
loss: 0.0019170131884045816 step took 22.47248148918152
output:
tensor([[  3.4638, -14.5782,  10.0514,   1.4316],
        [  2.5166, -14.9352,  10.3771,   2.4137],
        [  1.7586,   9.5291, -14.9346,   3.5422],
        [  3.7745, -14.9621,  10.3596,   1.1964],
        [  1.1212,   9.9691, -15.2567,   4.0700]], device='cuda:0',
       dtype=torch.float64, grad_fn=<AddmmBackward>) 
 labels:
tensor([[0, 0, 1, 0],
        [0, 0, 1, 0],
        [0, 1, 0, 0],
        [0, 0, 1, 0],
        [0, 1, 0, 0]], device='cuda:0', dtype=torch.uint8)
mel size: 24
sequence le

        [0, 0, 0, 1]], device='cuda:0', dtype=torch.uint8)
mel size: 24
sequence length: 400
output shape: torch.Size([5, 396, 512])
mel size: 512
sequence length: 396
output shape: torch.Size([5, 392, 512])
mel size: 512
sequence length: 392
output shape: torch.Size([5, 386, 512])
mel size: 512
sequence length: 386
output shape: torch.Size([5, 385, 512])
mel size: 512
sequence length: 385
output shape: torch.Size([5, 384, 1500])
loss: 0.006370145245278636 step took 22.48775625228882
output:
tensor([[  1.7583,  11.8232, -17.2178,   3.4288],
        [  5.3112,  -1.2064,  -4.4870,   0.4125],
        [  3.4027, -11.8221,   7.3484,   1.3751],
        [  1.5969,  12.0362, -17.4722,   3.6405],
        [  1.3971,  11.0435, -16.3211,   3.7289]], device='cuda:0',
       dtype=torch.float64, grad_fn=<AddmmBackward>) 
 labels:
tensor([[0, 1, 0, 0],
        [1, 0, 0, 0],
        [0, 0, 1, 0],
        [0, 1, 0, 0],
        [0, 1, 0, 0]], device='cuda:0', dtype=torch.uint8)
mel size: 24
sequence len

        [0, 0, 1, 0]], device='cuda:0', dtype=torch.uint8)
mel size: 24
sequence length: 400
output shape: torch.Size([5, 396, 512])
mel size: 512
sequence length: 396
output shape: torch.Size([5, 392, 512])
mel size: 512
sequence length: 392
output shape: torch.Size([5, 386, 512])
mel size: 512
sequence length: 386
output shape: torch.Size([5, 385, 512])
mel size: 512
sequence length: 385
output shape: torch.Size([5, 384, 1500])
loss: 0.010733586327569 step took 25.17533540725708
output:
tensor([[  1.5595,  10.9934, -16.2066,   3.4902],
        [  5.9703,  -6.5444,   1.2802,  -0.6241],
        [ -0.6663,  -4.3269,  -0.5366,   5.9464],
        [  3.8472, -14.4419,  10.0481,   0.8944],
        [  4.4639,  -2.3455,  -3.1747,   1.1569]], device='cuda:0',
       dtype=torch.float64, grad_fn=<AddmmBackward>) 
 labels:
tensor([[0, 1, 0, 0],
        [1, 0, 0, 0],
        [0, 0, 0, 1],
        [0, 0, 1, 0],
        [1, 0, 0, 0]], device='cuda:0', dtype=torch.uint8)
mel size: 24
sequence length

KeyboardInterrupt: 

### Embedding

In [83]:
net_xvec = nn.Sequential(net1,net2,net3,net4,net5,SP,FC)

In [84]:
net_xvec.cuda()

Sequential(
  (0): TDNN()
  (1): TDNN()
  (2): TDNN()
  (3): TDNN()
  (4): TDNN()
  (5): StatsPooling()
  (6): FullyConnected(
    (hidden1): Linear(in_features=3000, out_features=512, bias=True)
    (hidden2): Linear(in_features=512, out_features=512, bias=True)
  )
)

 ### Testing

In [ ]:
net = torch.load('total_model')
net.eval()

In [ ]:
for p in list(net.parameters()):
    print (p.shape)

In [ ]:
list(net[0].parameters())

In [ ]:
for p in list(net.parameters()):
    print (p)

In [ ]:
net.cuda()

In [75]:
wav_path = r"D:\ml+dl+dsp\xvector\train_cleaned\Macron.wav"
max_total_context_test = 400
_min,_max = float('inf'),-float('inf')
X_test = []
rate, wav = wavfile.read(wav_path)
for chunked_wav in tqdm(chunks(wav,int(len(wav)/4))):
    X_sample = mfcc(chunked_wav,samplerate= rate,numcep=24
           ,nfilt=26,nfft=1024)
    _min = min(np.amin(X_sample),_min)
    _max = max(np.amax(X_sample),_max)
    for chunked_X_sample in list(chunks(X_sample,max_total_context_test)):
            if len(chunked_X_sample) == max_total_context_test:
                X_test.append(chunked_X_sample)
X_test = (X_test - _min) / (_max-_min)


C:\Users\Daniel\.conda\envs\pytorch\lib\site-packages\scipy\io\wavfile.py:273: WavFileWarning: Chunk (non-data) not understood, skipping it.
  WavFileWarning)
0it [00:00, ?it/s]WARNING:root:frame length (1103) is greater than FFT size (1024), frame will be truncated. Increase NFFT to avoid.
1it [00:05,  5.62s/it]WARNING:root:frame length (1103) is greater than FFT size (1024), frame will be truncated. Increase NFFT to avoid.
2it [00:08,  4.75s/it]WARNING:root:frame length (1103) is greater than FFT size (1024), frame will be truncated. Increase NFFT to avoid.
3it [00:11,  4.26s/it]WARNING:root:frame length (1103) is greater than FFT size (1024), frame will be truncated. Increase NFFT to avoid.
4it [00:14,  3.90s/it]WARNING:root:frame length (1103) is greater than FFT size (1024), frame will be truncated. Increase NFFT to avoid.
5it [00:14,  2.92s/it]


In [ ]:
torch.tensor(X_test).shape

In [ ]:
del net

In [ ]:
net.to('cpu')

In [85]:
output= net(torch.tensor(X_test[100:120]).cuda())
for out in output:
    print (clases[torch.argmax(out).cuda()])

mel size: 24
sequence length: 400
output shape: torch.Size([70, 396, 512])
mel size: 512
sequence length: 396
output shape: torch.Size([70, 392, 512])
mel size: 512
sequence length: 392
output shape: torch.Size([70, 386, 512])
mel size: 512
sequence length: 386
output shape: torch.Size([70, 385, 512])
mel size: 512
sequence length: 385


RuntimeError: CUDA out of memory. Tried to allocate 308.00 MiB (GPU 0; 4.00 GiB total capacity; 2.31 GiB already allocated; 300.66 MiB free; 209.59 MiB cached)

In [89]:
X_test[:70].shape

(70, 400, 24)

In [106]:
torch.cuda.empty_cache()

In [107]:
torch.cuda.max_memory_allocated()

2485554688

In [108]:
torch.cuda.reset_max_memory_allocated()

In [109]:
torch.cuda.reset_max_memory_cached()

In [110]:
del X_test

NameError: name 'X_test' is not defined

In [61]:
X_test.shape

(116, 400, 24)

In [22]:
150 * 44100

6615000

In [31]:
X_test.shape

(116, 400, 24)

In [70]:
torch.cuda.empty_cache()

In [ ]:
torch.argmax(output[0])

In [ ]:
torch.max(labels, 1)

In [ ]:
labels[0]

In [ ]:
output

In [ ]:
loss

In [ ]:
loss = criterion(output, torch.max(labels, 1)[1])


loss: 1.394379672011826 step took 183.8471429347992

In [ ]:
torch.tensor([X[0]]).shape

In [ ]:
for i, data in enumerate(trainloader, 0):
    # get the inputs; data is a list of [inputs, labels]
    inputs, labels = data

In [ ]:
len(np.array([X[0],X[1]]).shape)

In [ ]:
output= net(inputs)#.to(device)


In [ ]:
net1(inputs).shape

In [ ]:
for i, data in enumerate(trainloader, 0):
    a = time.time()
    # get the inputs; data is a list of [inputs, labels]
    inputs, labels = data
    break

In [ ]:
net1(inputs.cuda()).shape

In [ ]:
mean = net1(inputs.cuda()).mean(dim=1)

In [ ]:
mean.shape

In [ ]:
inputs.shape

In [ ]:
output.reshape((2,1,4))

In [ ]:
(torch.max(labels, 1)[1]).shape

In [ ]:
output.shape

In [ ]:
out

In [ ]:
criterion(torch.tensor([]))

In [ ]:
loss = criterion(output, torch.max(labels, 1)[1])
loss.backward()
optimizer.step()

In [ ]:
torch.tensor([[0,0,0,999],[0,0,0,999]])

In [ ]:
mean = output.mean(dim=1)
std = output.std(dim=1)
# ms = torch.cat((mean,std),dim=1)

In [ ]:
nn.

In [ ]:
(mean*396).shape

In [ ]:
mean[2].shape

In [ ]:
output[::1,].shape

In [ ]:
output[:,:1,] - mean[1]

In [ ]:
output.std(dim=1).shape

In [ ]:
Final(FC(ms)).shape

In [ ]:
SP(output).shape

In [ ]:
inputs.shape

In [ ]:
net.load_state_dict

In [ ]:
inputs

In [ ]:
torch._C._cuda_init()

In [ ]:
output.reshape((1,4))

In [ ]:
criterion = nn.CrossEntropyLoss()
criterion(output.reshape((1,4)), torch.max(labels, 1)[1])

In [ ]:
output = net1(inputs.cuda())

In [ ]:
F.normalize(output,dim=1).shape

In [ ]:
o=torch.tensor([[[0,2,4],[4,5,6],[7,8,9]],[[1,2,3],[4,5,6],[7,8,9]],[[1,2,3],[4,5,6],[7,8,9]]]).float()

In [ ]:
mean = output.mean(dim=2)
std = output.std(dim=2)
print (mean)
print ()
print (std)

In [ ]:
mean.shape

In [ ]:
((output-mean.transpose(0,1)))#.transpose(0,2) /  std.transpose(0,1)).transpose(0,2)

In [ ]:
(o-mean.transpose(0,1)).transpose(1,2) /  std.transpose(0,1)

In [ ]:
F.normalize(o.float(),dim=2)

In [ ]:
o

In [ ]:
torch.tensor([0,0,1,0]).reshape((1,4))

In [ ]:
labels

In [ ]:
criterion(torch.tensor([0,0,1,0]).reshape((1,4)).float(), torch.max(labels, 1)[1])

In [ ]:
torch.cat( (output.mean(dim=1)[0],output.std(dim=1)[0] )).shape

In [ ]:
output

In [ ]:
F.softmax(output).shape

In [ ]:
L2 = nn.Linear(512,512).double()

In [ ]:
L2(F.relu(output))

In [ ]:
output.mean(dim=1)

In [ ]:
output[0].std(dim=0)

In [ ]:
output[0][0]

In [ ]:
output[0][1]

In [ ]:
output

In [ ]:
output[0][2]

In [ ]:
output[0][3]